## Cubic interpolation
Let's assume that we want to interpolate the value in between two given points. And we are not interested in linear interpolation, but would like to fit a polynomial of degree 3. The general equation for the polynomial of degree 3 is:
$$
    f(x) = ax^3 + bx^2 + cx + d
$$
 Let's also assume that we want to interpolate for values $x \in [0,1]$, where f(0) and f(1) are given.
 So to fit f(x) we need to find the coefficients (a,b,c,d). For now we have 4 unknowns (a,b,c,d) and 2 equations (for f(0) and f(1)), which gives us underdefined system of linear equations. (Note: the unknowns are a,b,c and d and they are linear).
 
 **Hypothesis 1**  To compensate for this problem we take into account the first derivatives in given points f'(0) and f'(1).
 
**Hypothesis 2** My natural solution, however, would be to add more points to build the system. Somehow this is not a solution I found (???)

Following hypothesis 1 for now.
$$
    f'(x) = 3ax^2 + 2bx + c
$$

Then we can obtain the following system of equations:   
$
    f(0) = d \\
    f(1) = a + b + c +d \\
    f'(0) = c \\
    f'(1) = 3a + 2b + c
$

Solving for a, b, c, d:  
\begin{eqnarray}
    d &=& f(0) \\
    c &=& f'(0) \\
    b &=& 3f(1)-2f'(0) - 3f(0) - f'(1) \\
    a &=& -2f(1) + f'(0) + 2f(0) + f'(1)
\end{eqnarray}

Ok, found polynomial coefficients. But how do we know the derivatives given just f(0) and f(1)? This is a point in time where we need two extra points and the knowledge of finite differences :smile:

Let's consider the following example. We are given points $[-1,p_0]$, $[0,p_1]$, $[1,p_2]$ and $[2, p_3]$, where $x,f(x)]$. and we want to interpolate between $[0, p_1]$ and $[1, p_2]$.
Then,  
$
    f(0) = p_1 \\
    f(1) = p_2 \\
    f'(0) = (p_2 - p_0) / 1-(-1)  \\
    f'(1) = (p_3 - p_1) / 2  \\
$

the following holds:
\begin{eqnarray}
    a &=& -2p_2 + 0.5(p_2 - p_0) + 2p_1 + 0.5(p_3 - p_1) \\
    b &=& 2p_2 + p_0 - 3p_1- 0.5(p_3 - p_1) \\
    c &=& 0.5(p_2 - p_0) \\
    d &=& p_1
\end{eqnarray}

**Warning** This only works if your x is between 0 and 1 and all the points are located at distance 1 from each other in x axis.

In [7]:
## Code for testing the cubic interpolation
import numpy as np

p_x = np.array([-1, 0, 1, 2])
p_fx = np.array([2, 5, 3,4])

a = -2 * p_fx[2] + 0.5 * (p_fx[2] - p_fx[1]) + 2*p_fx[1] + 0.5* (p_fx[3] - p_fx[1]);
b = 2 * p_fx[2]  + p_fx[0] - 3*p_fx[1] - 0.5*(p_fx[3] - p_fx[1]);
c = 0.5 * (p_fx[2]- p_fx[0])
d = p_fx[1]

x = 1;
f_x = a *x*x*x + b*x*x + c*x +d;
print(f_x)

1.5
